In [ ]:
import argparse
import os
import torch
import numpy as np
import cv2
import sys
import shutil
import matplotlib.pyplot as plt

from attrdict import AttrDict
import pandas as pd

from torch.autograd import Variable
from sgan.data.loader import data_loader
from sgan.models import CNNLSTM1, CNNLSTM2, CNNLSTM3

model_path = "./models/cnnlstm2-vgg16-lausanne-noembedder_with_model.pt" 
dataset = "./datasets/lausanne"

def create_folder(directory, renew=False):
    
    # if directory doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # if already exist, delete and re-create
    elif os.path.exists(directory) and renew:
        shutil.rmtree(directory)
        os.makedirs(directory)

def get_classifier(checkpoint):
        
    args = AttrDict(checkpoint['args'])  
    classifier = CNNLSTM2(
        embedding_dim=args.embedding_dim,
        h_dim=args.h_dim,
        dropout=0.0,
        grad=True)
    classifier.load_state_dict(checkpoint['best_state'])
    return classifier

def save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, result):
                
    create_folder(os.path.join(os.getcwd(), "results", result, os.path.basename(pedestrian_foldernames[0][0])))    
    for g,foldername,filename in zip(gradients, pedestrian_foldernames[0], pedestrian_filenames[0]):
        print(os.path.join(os.getcwd(), "results", result, os.path.basename(foldername), filename))
        cv2.imwrite(os.path.join(os.getcwd(), "results", result, os.path.basename(foldername), filename), np.uint8(g*255))

def guidedbackprop(args, loader, classifier):
       
    classifier.train()        
    for batch in loader:

        (pedestrian_crops, decision_true, pedestrian_foldernames, pedestrian_filenames) = batch
        
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000133")
        # (str(pedestrian_foldernames[0][0]) == "crops/Ouchy-2-Left/0000000159")
        if(1):
            
            decision_pred_scores = classifier(pedestrian_crops, input_as_var=True, classify_every_timestep=False)        
            if(torch.cuda.is_available()):
                decision_onehot = torch.cuda.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
            else:
                decision_onehot = torch.FloatTensor(1, decision_pred_scores.size()[-1]).zero_()
            decision_onehot[0][decision_pred_scores.max(1)[1]] = 1
            if(torch.cuda.is_available()):
                decision_onehot = decision_onehot.cuda()
            else:
                decision_onehot = decision_onehot
                
            # backprop
            classifier.zero_grad()
            decision_pred_scores.backward(gradient=decision_onehot)
            gradients = classifier.gradients.cpu().numpy()

            # normalize each image
            gradients_min = np.amin(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
            gradients_max = np.amax(np.reshape(gradients, newshape=(np.shape(gradients)[0],-1)), 1)
            gradients -= gradients_min[:,None,None,None]
            gradients /= gradients_max[:,None,None,None] - gradients_min[:,None,None,None]
            gradients = np.transpose(gradients, (0, 2, 3, 1))

            # save images to folder
            decision_true = decision_true.item()
            decision_pred = decision_pred_scores.max(1)[1].item()

            # tn
            if(decision_pred == 0 and decision_true == 0):
                save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "tn")            
            # fn
            if(decision_pred == 0 and decision_true == 1):
                save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "fn")            
            # fp
            if(decision_pred == 1 and decision_true == 0):
                save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "fp")            
            # tp
            if(decision_pred == 1 and decision_true == 1):
                save_gradients(gradients, pedestrian_foldernames, pedestrian_filenames, "tp")
            
def main(args):
            
    # create results folder if it does not exist
    # - used for guidedbackprop
    results_root = os.path.join(os.getcwd(), "results")
    create_folder(results_root, renew=False)
    
    # model path
    path = args.model_path
    print("model path ", path)

     # get the model
    checkpoint = torch.load(path, map_location='cpu')
    classifier = get_classifier(checkpoint)

    # generate the arguments
    _args = AttrDict(checkpoint['args']) 
    _args.batch_size = 1
    print(_args)

    # get the dataset
    val_path = os.path.join(args.dataset, "val") 
    dset, loader = data_loader(_args, val_path, "val")
    
    print('Running Guided Backprop')
    guidedbackprop(_args, loader, classifier)
        
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_path', type=str)
    parser.add_argument('--dset_type', default='val', type=str)

    parser.model_path = model_path
    parser.dataset = dataset #"./datasets/lausanne" #"./datasets/riponne" lausanne
    args = parser
    images_dict, prediction_dict = main(args)